In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
punch = pd.read_csv('data/punch.csv', header = None)
flex = pd.read_csv('data/flex.csv', header = None)
print(punch)
print(flex)

         0     1     2      3      4      5
0     1.96  0.62  0.94  69.58  98.39  55.54
1     1.84  0.68  0.87  77.94  30.09  55.24
2     1.53  0.71  0.81  73.18 -22.10  40.71
3     1.21  0.71  0.72  52.25 -39.25  19.35
4     0.98  0.69  0.62  20.39 -35.34  -0.92
...    ...   ...   ...    ...    ...    ...
6995 -0.31  0.96 -0.15   0.79   0.18   2.32
6996 -0.36  0.96 -0.13   2.50  -1.04   1.04
6997 -0.36  0.95 -0.14   2.14   0.00   0.49
6998 -0.37  0.95 -0.15   0.43   0.67   1.77
6999 -0.37  0.95 -0.16  -1.34  -1.34   0.67

[7000 rows x 6 columns]
         0     1     2       3       4       5
0    -0.49  1.55 -1.63 -216.99  367.44 -317.70
1    -0.90 -1.98 -1.87 -204.29  170.96 -265.88
2    -1.01 -1.98 -1.87 -214.42  -27.53 -171.76
3    -1.09 -1.98 -1.87 -206.00 -166.39  -93.57
4    -1.00 -1.98 -1.70 -153.81 -160.71  -39.12
...    ...   ...   ...     ...     ...     ...
6995 -0.77  0.62  0.32    9.28    5.92    1.28
6996 -0.84  0.62  0.31   17.33   -4.27   -3.91
6997 -0.90  0.61  0.29  

In [3]:
SAMPLES_PER_GESTURE = 70

In [4]:
def processData(d, v):
    dataX = np.empty([0,SAMPLES_PER_GESTURE*6])
    dataY = np.empty([0])

    data  = d.values
    dataNum = data.shape[0] // SAMPLES_PER_GESTURE


    for i in tqdm(range(dataNum)):
        tmp = []
        for j in range(SAMPLES_PER_GESTURE):
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][0] + 4.0) / 8.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][1] + 4.0) / 8.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][2] + 4.0) / 8.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][3] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][4] + 2000.0) / 4000.0]
            tmp += [(data[i * SAMPLES_PER_GESTURE + j][5] + 2000.0) / 4000.0]

        tmp = np.array(tmp)

        tmp = np.expand_dims(tmp, axis = 0)

        dataX = np.concatenate((dataX, tmp), axis = 0)
        dataY = np.append(dataY, v)

    return dataX, dataY

In [5]:
punchX, punchY = processData(punch, 0)
flexX, flexY = processData(flex, 1)
dataX = np.concatenate((punchX, flexX), axis = 0)
dataY = np.concatenate((punchY, flexY), axis = 0)

100%|██████████| 100/100 [00:00<00:00, 1448.20it/s]


In [6]:
permutationTrain = np.random.permutation(dataX.shape[0])
print(permutationTrain)

[154  55  96 174 167 131  27  25 175   7 182   6 125 100  78 160 189  35
 196  49   4 165  97  72 148  68  37  19  34  69  53  56 185 129  51  50
 172  61 106  43 179 147 134 135 176  38  57 121   3 149 136  12  60  54
 183  86  85  33 158 195  64 142  47  62   5  13 193 105 145  79 156 132
  44 128  32 168  73  95  93 117 150  21 120  41  46  99 180 177  74 198
 110 157  82  45  14  90 123 112   0  29  77  91 194 102 114 119 191  84
 143  87 188 115  66 109 137  83  10  31 184  89  24 163 104  92 171  48
 118  80  18  11 130  30   1  58 166 192  70   8  23 190  15  81 111  52
 151 178 107  75  36  88 133 169 101 197   2  65   9 140  71  40 103 170
 173 159  28  98 126 139  26  20 164  42 186 144 153 138  39 127 122 141
  59  17 187 152 113 161  22  63  67 116 199 181  94  16 162 155  76 146
 108 124]


In [7]:
dataX = dataX[permutationTrain]
dataY = dataY[permutationTrain]
print(dataY)

[1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1.
 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1.
 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1.
 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 0. 0. 1. 1. 0. 1. 1. 1.]


In [8]:
vfoldSize = int(dataX.shape[0]/100*20)

xTest = dataX[0:vfoldSize]
yTest = dataY[0:vfoldSize]

xTrain = dataX[vfoldSize:dataX.shape[0]]
yTrain = dataY[vfoldSize:dataY.shape[0]]

In [9]:
model = keras.Sequential()
model.add(keras.layers.Dense(32, input_shape =(6*SAMPLES_PER_GESTURE,), activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))
adam = keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,
              metrics=['sparse_categorical_accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                13472     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 14,034
Trainable params: 14,034
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(xTrain, yTrain, batch_size=1, validation_data=(xTest, yTest), epochs=200, verbose=1)

Train on 160 samples, validate on 40 samples
Epoch 1/200
160/160 [==============================] - 1s 9ms/sample - loss: 0.7056 - sparse_categorical_accuracy: 0.5250 - val_loss: 0.7012 - val_sparse_categorical_accuracy: 0.4250
Epoch 2/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.6945 - sparse_categorical_accuracy: 0.5063 - val_loss: 0.6591 - val_sparse_categorical_accuracy: 0.7000
Epoch 3/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.6408 - sparse_categorical_accuracy: 0.6313 - val_loss: 0.5928 - val_sparse_categorical_accuracy: 0.7500
Epoch 4/200
160/160 [==============================] - 1s 3ms/sample - loss: 0.6005 - sparse_categorical_accuracy: 0.6187 - val_loss: 0.5986 - val_sparse_categorical_accuracy: 0.5750
Epoch 5/200
160/160 [==============================] - 1s 4ms/sample - loss: 0.5449 - sparse_categorical_accuracy: 0.7375 - val_loss: 0.6080 - val_sparse_categorical_accuracy: 0.6500
Epoch 6/200
160/160 [===================